In [16]:
import dask.dataframe as dd
df = dd.read_csv("../VoxelsDataset/wavenumberVoxels/*").compute().iloc[:, 2:]
df.head()

,index,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330
0,-117.413551,-117.414613,-117.417829,-117.423202,-117.430734,-117.440428,-117.452289,-117.466324,-117.482539,-117.500943,...,False,False,False,False,False,False,False,False,False,False
1,-113.881322,-113.882395,-113.885622,-113.891006,-113.898548,-113.908253,-113.920126,-113.934171,-113.950398,-113.968812,...,False,False,False,False,False,False,False,False,False,False
2,-123.985723,-123.986806,-123.990043,-123.995437,-124.002989,-124.012704,-124.024586,-124.038642,-124.054878,-124.073302,...,False,False,False,False,False,False,False,False,False,False
3,-114.424147,-114.425230,-114.428468,-114.433861,-114.441414,-114.451129,-114.463012,-114.477068,-114.493304,-114.511729,...,False,False,False,False,False,False,False,False,False,False
4,-131.450275,-131.451352,-131.454583,-131.459971,-131.467517,-131.477226,-131.489102,-131.503152,-131.519382,-131.537801,...,False,False,False,False,False,False,False,False,False,True


In [17]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:401]
Y = df.iloc[:,401:]

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.7, random_state = 42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.7, random_state = 42)


In [18]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_val = ss.transform(x_val)
x_test = ss.transform(x_test)


In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD, Adam


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=50, verbose=1,)
early_stop = EarlyStopping(monitor='val_loss', patience=61, verbose=1, restore_best_weights=True)
opt = Adam()

model = Sequential()
model.add(Dense(1000, input_dim=401, kernel_initializer='normal', activation='relu'))
model.add(Dense(500, kernel_initializer='normal', activation='relu'))
model.add(Dense(1331, kernel_initializer='normal', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'], )
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1000)              402000    
_________________________________________________________________
dense_8 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_9 (Dense)              (None, 1331)              666831    
Total params: 1,569,331
Trainable params: 1,569,331
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = model.fit(x=x_train, y=y_train, validation_data = (x_val, y_val), epochs=1000, verbose=1, callbacks=[early_stop, reduce_lr])

=========] - 1s 593us/step - loss: 0.0982 - accuracy: 0.9562 - val_loss: 0.0931 - val_accuracy: 0.9589
Epoch 496/1000
1523/1523 [==============================] - 1s 577us/step - loss: 0.0979 - accuracy: 0.9563 - val_loss: 0.0999 - val_accuracy: 0.9550
Epoch 497/1000
1523/1523 [==============================] - 1s 572us/step - loss: 0.0956 - accuracy: 0.9577 - val_loss: 0.1098 - val_accuracy: 0.9495
Epoch 498/1000
1523/1523 [==============================] - 1s 596us/step - loss: 0.0982 - accuracy: 0.9560 - val_loss: 0.0943 - val_accuracy: 0.9590
Epoch 499/1000
1523/1523 [==============================] - 1s 826us/step - loss: 0.0930 - accuracy: 0.9588 - val_loss: 0.1080 - val_accuracy: 0.9516
Epoch 500/1000
1523/1523 [==============================] - 1s 660us/step - loss: 0.0988 - accuracy: 0.9566 - val_loss: 0.0937 - val_accuracy: 0.9583
Epoch 501/1000
1523/1523 [==============================] - 1s 601us/step - loss: 0.0963 - accuracy: 0.9576 - val_loss: 0.0946 - val_accuracy: 0.95